Imports

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import matplotlib as plt
import torch

In [2]:
torch.cuda.is_available()

False

Download datasets

In [7]:
# add download of ModelNet40 in data/ModelNet40
print('Downloading ModelNet40...')
!wget modelnet.cs.princeton.edu/ModelNet40.zip -P data
print('Extracting ModelNet40...')
!unzip -q data/ModelNet40.zip -d data
!rm data/ModelNet40.zip
print('ModelNet40 done')

# add download of 7Scenes in data/tsdf
print('Downloading 7Scenes...')
!wget download.microsoft.com/download/2/8/5/28564B23-0828-408F-8631-23B1EFF1DAC8/tsdf.zip -P data
print('Extracting 7Scenes...')
!unzip -q data/tsdf.zip -d data
!rm data/tsdf.zip
print('7Scenes done')

/bin/sh: 1: wget: not found
Extracting ModelNet40...
/bin/sh: 1: unzip: not found
ModelNet40 done


Run simple configuration

In [7]:
# run simple configuration

Visualize results

In [8]:
# add visualization